# Format For Paper

In [1]:
import influxdb_client
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy
import json

os.chdir('../')
print(os.getcwd())

from run import influx_cfg
print(influx_cfg.bucket)

/Volumes/external_drive/Code/MV2/Demo
modicumdb


In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# Message Time

In [24]:
def load_influx(num_customers='10', num_suppliers='10', supplier_behavior='lazy'):
    # load data
    file_path = os.path.join(os.getcwd(), "demo_results", "raw_influx", f"{num_customers}-{num_suppliers}-{supplier_behavior}.csv")
    df = pd.read_csv(file_path)
    df['_time'] = pd.to_datetime(df['_time'])
    
    # filter data
    state = df[df['_field']=='state']
    state = state[state['entity'].isin(['customer', 'supplier'])]
    completed_uuids = state[state['_value']=='11']['uuid'].unique().tolist()
    print(f"Number of completed uuids: {len(completed_uuids)}")
    state = state[state['uuid'].isin(completed_uuids)]
    
    alloc = df[df['_field']=='allocation']
    alloc = alloc[alloc['uuid'].isin(completed_uuids)]
    return df, state, alloc


In [25]:
num_customers='20'
num_suppliers='20'
supplier_behavior='nominal'

df, state, alloc = load_influx(num_customers=num_customers, num_suppliers=num_suppliers, supplier_behavior=supplier_behavior)
print(len(state), len(alloc))
df.head(2)

Number of completed uuids: 40
320 240


,time_ms,_measurement,_field,_value,uuid,_time,entity
0,0.0,Events,state,1,supplier_f641c48d95b148c1a8d9dee67f0ecd79_offer_0,2021-11-19 22:18:15.450189+00:00,supplier
1,0.0,Events,sender,supplier_f641c48d95b148c1a8d9dee67f0ecd79,supplier_f641c48d95b148c1a8d9dee67f0ecd79_offer_0,2021-11-19 22:18:15.450189+00:00,supplier


In [33]:
proc_time = []
for alloc_id in alloc['_value'].unique():
    print(alloc_id)
    custid = alloc[(alloc['_value']==alloc_id) & (alloc['entity']=='customer')].iloc[0]['uuid']
    supid = alloc[(alloc['_value']==alloc_id) & (alloc['entity']=='supplier')].iloc[0]['uuid']
    
    end_time = state[(state['uuid']==custid) & (state['_value']=='5')].iloc[0]['_time']
    start_time = state[(state['uuid']==supid) & (state['_value']=='5')].iloc[0]['_time']
    time_diff = (end_time-start_time).total_seconds()
    proc_time.append(time_diff)
print(proc_time)

allocation_0
allocation_1
allocation_2
allocation_3
allocation_4
allocation_5
allocation_6
allocation_7
allocation_8
allocation_9
allocation_10
allocation_11
allocation_12
allocation_13
allocation_14
allocation_15
allocation_16
allocation_17
allocation_18
allocation_19
[53.441454, 39.411365, 56.838424, 41.686925, 58.033462, 55.09144, 31.164322, 57.04085, 37.128318, 57.821278, 31.139466, 56.125505, 56.424192, 57.290033, 54.825187, 37.670345, 59.28422, 37.610104, 54.933484, 55.244774]


In [11]:
state.head()

,time_ms,_measurement,_field,_value,uuid,_time,entity
2,24.245,Events,state,1,customer_3132ddea002f455580aa475f30906ff9_offer_0,2021-11-19 22:18:15.474434+00:00,customer
4,26.101,Events,state,1,customer_8b2f56af585c4ca7a1cc91ba9297cd62_offer_0,2021-11-19 22:18:15.476290+00:00,customer
6,28.300,Events,state,1,customer_f0986368af8e4c2891916e363de3f573_offer_0,2021-11-19 22:18:15.478489+00:00,customer
9,36.654,Events,state,1,supplier_14fd0f14b0c04adca3abaa3bddeeb39b_offer_0,2021-11-19 22:18:15.486843+00:00,supplier
12,101.339,Events,state,1,customer_07ec8b82c2234b9db4c45edcc05a43fd_offer_0,2021-11-19 22:18:15.551528+00:00,customer


In [10]:
alloc.head()

,time_ms,_measurement,_field,_value,uuid,_time,entity
306,10451.205,Events,allocation,allocation_0,customer_07ec8b82c2234b9db4c45edcc05a43fd_offer_0,2021-11-19 22:18:25.901394+00:00,customer
310,10459.236,Events,allocation,allocation_0,supplier_14fd0f14b0c04adca3abaa3bddeeb39b_offer_1,2021-11-19 22:18:25.909425+00:00,supplier
315,10496.466,Events,allocation,allocation_1,supplier_31a1aa69b90449c684ac971c324db933_offer_1,2021-11-19 22:18:25.946655+00:00,supplier
319,10497.450,Events,allocation,allocation_1,customer_81b05de5c8b3447392a8c2084a54669b_offer_0,2021-11-19 22:18:25.947639+00:00,customer
324,10540.587,Events,allocation,allocation_2,customer_955cde3e0e204653a5b676368788ba6b_offer_0,2021-11-19 22:18:25.990776+00:00,customer


In [12]:
alloc[alloc['_value']=='allocation_0']

,time_ms,_measurement,_field,_value,uuid,_time,entity
306,10451.205,Events,allocation,allocation_0,customer_07ec8b82c2234b9db4c45edcc05a43fd_offer_0,2021-11-19 22:18:25.901394+00:00,customer
310,10459.236,Events,allocation,allocation_0,supplier_14fd0f14b0c04adca3abaa3bddeeb39b_offer_1,2021-11-19 22:18:25.909425+00:00,supplier
335,10609.499,Events,allocation,allocation_0,customer_07ec8b82c2234b9db4c45edcc05a43fd_offer_0,2021-11-19 22:18:26.059688+00:00,customer
337,10612.784,Events,allocation,allocation_0,supplier_14fd0f14b0c04adca3abaa3bddeeb39b_offer_1,2021-11-19 22:18:26.062973+00:00,supplier
732,61702.300,Events,allocation,allocation_0,supplier_14fd0f14b0c04adca3abaa3bddeeb39b_offer_1,2021-11-19 22:19:17.152489+00:00,supplier
757,115143.754,Events,allocation,allocation_0,customer_07ec8b82c2234b9db4c45edcc05a43fd_offer_0,2021-11-19 22:20:10.593943+00:00,customer
933,878099.451,Events,allocation,allocation_0,customer_07ec8b82c2234b9db4c45edcc05a43fd_offer_0,2021-11-19 22:32:53.549640+00:00,customer
983,885388.220,Events,allocation,allocation_0,supplier_14fd0f14b0c04adca3abaa3bddeeb39b_offer_1,2021-11-19 22:33:00.838409+00:00,supplier
988,885802.075,Events,allocation,allocation_0,customer_07ec8b82c2234b9db4c45edcc05a43fd_offer_0,2021-11-19 22:33:01.252264+00:00,customer
989,885809.474,Events,allocation,allocation_0,customer_07ec8b82c2234b9db4c45edcc05a43fd_offer_0,2021-11-19 22:33:01.259663+00:00,customer


# State Duration

In [14]:
def load_influx(num_customers='10', num_suppliers='10', supplier_behavior='lazy'):
    # load data
    file_path = os.path.join(os.getcwd(), "demo_results", "raw_influx", f"{num_customers}-{num_suppliers}-{supplier_behavior}.csv")
    df = pd.read_csv(file_path)
    df['_time'] = pd.to_datetime(df['_time'])
    
    # filter data
    state = df[df['_field']=='state']
    state = state[state['entity'].isin(['customer', 'supplier'])]
    completed_uuids = state[state['_value']=='11']['uuid'].unique().tolist()
    print(f"Number of completed uuids: {len(completed_uuids)}")
    state = state[state['uuid'].isin(completed_uuids)]
    
    # process
    result = {'state_id': [], 'state': [], 'entity': [], 'uuid': [], 'time (s)': [], 'start_timestamp': [], 'end_timestamp': []}
    state_pairs = [[1,4], [4,5], [7,11]]
    state_names = ['a', 'b', 'c']
    for uuid in state['uuid'].unique():
        i = 0
        for state_pair in state_pairs:
            try:
                start_time = state.loc[(state['uuid']==uuid) & (state['_value']==str(state_pair[0])), 'time_ms'].iloc[0]
                end_time = state.loc[(state['uuid']==uuid) & (state['_value']==str(state_pair[1])), 'time_ms'].iloc[0]
                time_diff = (end_time - start_time)/1000
                
                start_timestamp = state.loc[(state['uuid']==uuid) & (state['_value']==str(state_pair[0])), '_time'].iloc[0]
                end_timestamp = state.loc[(state['uuid']==uuid) & (state['_value']==str(state_pair[1])), '_time'].iloc[0]
            except:
                time_diff = None
                start_timestamp = None
                end_timestamp = None
            result['state_id'].append(f"{state_pair[0]}")
            result['state'].append(state_names[i])
            result['entity'].append(f"{uuid.split('_')[0]}")
            result['uuid'].append(uuid)
            #result['time (s)'].append(time_diff/1000)
            result['time (s)'].append(time_diff)
            result['start_timestamp'].append(start_timestamp)
            result['end_timestamp'].append(end_timestamp)
            result
            i += 1
    df_plot = pd.DataFrame(result)
    return df_plot

def format_sim(df_plot):
    result = {}
    for entity in ['customer', 'supplier']:
        for state in ['a', 'b', 'c']:
            temp = df_plot[(df_plot['entity']==entity) & (df_plot['state']==state)]
            if (state=='a') & (entity=='customer'):
                col = 'custalloc'
            elif (state=='a') & (entity=='supplier'):
                col = 'supalloc'
            elif (state=='b') & (entity=='customer'):
                col = 'prot'
            elif (state=='b') & (entity=='supplier'):
                col = 'supapp'
            elif (state=='c') & (entity=='customer'):
                col = 'custmed'
            elif (state=='c') & (entity=='supplier'):
                col = 'supmed'
            result[col] = temp['time (s)'].values.tolist()
    df = pd.DataFrame(result)
    return df

num_customers='20'
num_suppliers='20'
supplier_behavior='lazy'

df = load_influx(num_customers=num_customers, num_suppliers=num_suppliers, supplier_behavior=supplier_behavior)
df_plot = format_sim(df)

file_path = os.path.join(os.getcwd(), "demo_results", "latex", f"simsix.csv")
df_plot = df_plot.dropna()
df_plot.to_csv(file_path, index=False)
df_plot

Number of completed uuids: 40


,custalloc,prot,custmed,supalloc,supapp,supmed
0,10.422721,34.539285,168.671177,8.788454,6.158487,248.689436
1,10.774402,30.199542,249.262182,8.725408,5.598493,247.967081
2,10.066515,34.606278,168.716184,9.290704,4.237494,249.554640
3,10.611776,30.066235,247.562110,7.758069,6.905636,248.978513
4,10.087023,20.259716,230.553638,6.772203,5.841806,168.824339
5,10.305656,33.712211,171.896699,6.843139,6.601543,230.663724
6,10.026611,23.212272,228.088699,7.193245,6.733723,171.849828
7,10.196448,30.115083,248.578824,6.500672,8.153837,169.415164
8,10.015678,30.120810,247.753574,5.699559,5.362498,168.712219
9,9.654637,34.403474,169.473187,5.431104,5.306371,220.685389


# Resources

In [18]:
def fix_service(service):
    if 'supplier' in service:
        return 'supplier'
    elif 'customer' in service:
        return 'customer'
    elif 'mediator' in service:
        return 'mediator'
    elif 'verifier' in service:
        return 'verifier'
    elif 'allocator' in service:
        return 'allocator'
    else:
        return 'none'

def load_consumption(num_customers='10', num_suppliers='10', supplier_behavior='lazy'):
    file_path = os.path.join(os.getcwd(), "demo_results", "raw_consumption", f"{num_customers}-{num_suppliers}-{supplier_behavior}-consumption.csv")
    #file_path = os.path.join(os.getcwd(), "demo_results", "raw_usage", f"{num_customers}-{num_suppliers}-{supplier_behavior}-usage.csv")
    usage = pd.read_csv(file_path)
    
    usage['start_time'] = pd.to_datetime(usage['start_time'])
    usage['end_time'] = pd.to_datetime(usage['end_time'])
    usage['time_diff'] = usage.apply(lambda row: (row['end_time']-row['start_time']).total_seconds(), axis=1)

    service = []
    usage_val = []
    usage_unit = []
    for k, v in usage.iterrows():
        temp = json.loads(v['labels'])['labels']
        temp2 = {x['key']: x['value'] for x in temp}
        if 'io.kompose.service' in temp2.keys():
            service.append(temp2['io.kompose.service'])
        else:
            service.append('none')
        temp3 = json.loads(v['usage'])['usage']
        usage_val.append(float(temp3['amount']))
        usage_unit.append(temp3['unit'])

    usage['service'] = service
    print(len(usage[usage['service']=='none']))
    usage['service'] = usage['service'].apply(lambda x: fix_service(x))
    print(len(usage[usage['service']=='none']))
    usage['usage_total'] = usage_val
    usage['usage_unit'] = usage_unit
    usage['usage'] = usage.apply(lambda row: row['usage_total']/row['time_diff'], axis=1)
    usage = usage.drop(columns=['labels', 'project'])
    return usage

def time_sync(num_customers='10', num_suppliers='10', supplier_behavior='lazy'): 
    # load data
    file_path = os.path.join(os.getcwd(), "demo_results", "raw_influx", f"{num_customers}-{num_suppliers}-{supplier_behavior}.csv")
    df = pd.read_csv(file_path)
    df['_time'] = pd.to_datetime(df['_time'])
    
    start_time = df['_time'].min()
    end_time = df['_time'].max()
    time_diff = (end_time - start_time).total_seconds()
    print(start_time, end_time, time_diff)
    return start_time, end_time, time_diff

num_customers='20'
num_suppliers='10'
supplier_behavior='lazy'

usage = load_consumption(num_customers=num_customers, num_suppliers=num_suppliers, supplier_behavior=supplier_behavior)
usage.head(1)

1738
1738


,cluster_location,cluster_name,namespace,resource_name,sku_id,start_time,end_time,fraction,cloud_resource_size,usage,time_diff,service,usage_total,usage_unit
0,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 22:17:59+00:00,2021-11-19 22:59:24+00:00,0.000733,4,0.002934,2485.0,allocator,7.29,seconds


In [20]:
# get usage
num_customers='20'
num_suppliers='10'
supplier_behavior='lazy'

sims = [{'num_customers': '10', 'num_suppliers': '10', 'supplier_behavior': 'nominal', 'sim': 1, 'simname': 'simone'},
       {'num_customers': '20', 'num_suppliers': '10', 'supplier_behavior': 'nominal', 'sim': 2, 'simname': 'simtwo'},
       {'num_customers': '20', 'num_suppliers': '20', 'supplier_behavior': 'nominal', 'sim': 3, 'simname': 'simthree'},
       {'num_customers': '10', 'num_suppliers': '10', 'supplier_behavior': 'lazy', 'sim': 4, 'simname': 'simfour'},
       {'num_customers': '20', 'num_suppliers': '10', 'supplier_behavior': 'lazy', 'sim': 5, 'simname': 'simfive'},
       {'num_customers': '20', 'num_suppliers': '20', 'supplier_behavior': 'lazy', 'sim': 6, 'simname': 'simsix'}]

services = ['supplier', 'mediator', 'customer', 'verifier', 'allocator']
resource_names = ['memory', 'cpu']

usage = load_consumption(num_customers=num_customers, num_suppliers=num_suppliers, supplier_behavior=supplier_behavior)
results = []
for sim in sims:
    start_time, end_time, time_diff = time_sync(num_customers=sim['num_customers'], num_suppliers=sim['num_suppliers'], supplier_behavior=sim['supplier_behavior'])
    result = {'sim': sim['sim'], 'simname': sim['simname']}
    for service in services:
        for resource_name in resource_names:
            if sim['sim'] == 5:
                temp = usage[(usage['service']==service) & (usage['resource_name']==resource_name) & (usage['start_time'] <= start_time)]
            else:
                temp = usage[(usage['service']==service) & (usage['resource_name']==resource_name) & (usage['start_time'] <= start_time) & (usage['end_time'] >= end_time)]
            k = f"{service}{resource_name}"
            #val = temp['usage_total'].sum() / (len(temp) * time_diff)
            val = temp['usage'].mean()
            if resource_name == 'memory':
                val = val * 1e-6
            result[k] = val
    results.append(result)
df = pd.DataFrame.from_records(results)
df

1738
1738
2021-11-19 20:19:35.417737+00:00 2021-11-19 20:27:24.257230+00:00 468.839493
2021-11-20 01:38:47.995918+00:00 2021-11-20 01:52:55.358431+00:00 847.362513
2021-11-19 22:18:15.450189+00:00 2021-11-19 22:35:02.974478+00:00 1007.524289
2021-11-19 18:30:31.085070+00:00 2021-11-19 18:37:53.433708+00:00 442.348638
2021-11-20 18:58:06.465628+00:00 2021-11-20 19:08:56.086550+00:00 649.620922
2021-11-19 19:38:19.831357+00:00 2021-11-19 19:52:48.548085+00:00 868.716728


,sim,simname,suppliermemory,suppliercpu,mediatormemory,mediatorcpu,customermemory,customercpu,verifiermemory,verifiercpu,allocatormemory,allocatorcpu
0,1,simone,52.186790,0.003696,46.704589,0.002085,44.566960,0.002076,34.883456,0.001092,46.464329,0.002398
1,2,simtwo,60.074205,0.005204,43.006722,0.002365,44.562989,0.002994,29.760799,0.001095,46.575690,0.003350
2,3,simthree,49.748465,0.003409,41.543192,0.001924,43.709538,0.002372,29.998424,0.001042,47.450520,0.002934
3,4,simfour,48.316083,0.003269,1335.126488,0.014955,41.365019,0.002390,30.520828,0.001082,46.381390,0.002118
4,5,simfive,51.160431,0.003403,592.750494,0.011234,42.558054,0.002272,30.054580,0.001115,45.391165,0.002405
5,6,simsix,49.791143,0.003914,1886.748304,0.026921,43.166147,0.002847,29.724207,0.001159,45.571110,0.002459


In [21]:
file_path = os.path.join(os.getcwd(), "demo_results", "latex", f"resources.csv")
df.to_csv(file_path, index=False)

In [22]:

result = [[],[],[],[],[],[]]
sims = df.index.values.tolist()
columns = []
metrics = ['customercpu', 'suppliercpu', 'mediatorcpu']

for i in sims:
    columns.append(df.loc[i]['simname'])
    for metric in metrics:
        result[i].append(df.loc[i][metric])
result2 = {}
i=0
for col in columns:
    result2[col] = result[i]
    i += 1
result2 = pd.DataFrame(result2, columns=columns)
file_path = os.path.join(os.getcwd(), "demo_results", "latex", f"cpu.csv")
result2.to_csv(file_path, index=False)
result2

,simone,simtwo,simthree,simfour,simfive,simsix
0,0.002076,0.002994,0.002372,0.002390,0.002272,0.002847
1,0.003696,0.005204,0.003409,0.003269,0.003403,0.003914
2,0.002085,0.002365,0.001924,0.014955,0.011234,0.026921


In [23]:
result = [[],[],[],[],[],[]]
sims = df.index.values.tolist()
columns = []
metrics = ['customermemory', 'suppliermemory', 'mediatormemory']

for i in sims:
    columns.append(df.loc[i]['simname'])
    for metric in metrics:
        result[i].append(df.loc[i][metric])
result2 = {}
i=0
for col in columns:
    result2[col] = result[i]
    i += 1
result2 = pd.DataFrame(result2, columns=columns)
file_path = os.path.join(os.getcwd(), "demo_results", "latex", f"memory.csv")
result2.to_csv(file_path, index=False)
result2

,simone,simtwo,simthree,simfour,simfive,simsix
0,44.566960,44.562989,43.709538,41.365019,42.558054,43.166147
1,52.186790,60.074205,49.748465,48.316083,51.160431,49.791143
2,46.704589,43.006722,41.543192,1335.126488,592.750494,1886.748304


In [164]:
columns

['simone', 'simtwo', 'simthree', 'simfour']

In [17]:
service = 'mediator'
resource_name = 'cpu'
sim = sims[1]
start_time, end_time, time_diff = time_sync(num_customers=sim['num_customers'], num_suppliers=sim['num_suppliers'], supplier_behavior=sim['supplier_behavior'])
temp = usage[(usage['service']==service) & (usage['resource_name']==resource_name) & (usage['start_time'] <= start_time) & (usage['end_time'] >= end_time)]
temp

2021-11-20 01:38:47.995918+00:00 2021-11-20 01:52:55.358431+00:00 847.362513


,cluster_location,cluster_name,namespace,resource_name,sku_id,start_time,end_time,fraction,cloud_resource_size,usage,time_diff,service,usage_total,usage_unit


In [99]:
dir_path = os.path.join(os.getcwd(), "demo_results", "raw_influx")
files = os.listdir(dir_path)
files

['.DS_Store',
 '10-10-lazy.csv',
 '10-10-nominal.csv',
 '20-20-lazy.csv',
 '20-20-nominal.csv',
 'void']

In [ ]:
def get_cpu_ram(usage, num_customers='10', num_suppliers='10', supplier_behavior='lazy'):
    start_time, end_time, time_diff = time_sync(num_customers=num_customers, num_suppliers=num_suppliers, supplier_behavior=supplier_behavior)
    
    sim_usage = usage[(usage['service']=='sporadic-customer') & (usage['start_time'] <= start_time) & (usage['end_time'] >= end_time)]
    

In [101]:
x = usage[(usage['service']=='sporadic-customer') & (usage['start_time'] <= start_time) & (usage['end_time'] >= end_time)]
print(len(x))

40


In [87]:
usage['service'].unique()

array(['allocator-service', 'lazy-supplier', 'mediator-service',
       'sporadic-customer', 'verifier-service', 'none'], dtype=object)

In [89]:
usage[usage['service']=='sporadic-customer']

,cluster_location,cluster_name,namespace,resource_name,sku_id,start_time,end_time,fraction,cloud_resource_size,service,usage_val,usage_unit
28,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:29:52+00:00,2021-11-19 19:08:48+00:00,0.000388,4,sporadic-customer,3.630000e+00,seconds
29,us-central1-c,market-1,default,memory,F449-33EC-A5EF,2021-11-19 18:29:52+00:00,2021-11-19 19:08:48+00:00,0.002466,16792662016,sporadic-customer,9.672045e+10,byte-seconds
30,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:29:53+00:00,2021-11-19 19:09:00+00:00,0.000604,4,sporadic-customer,5.670000e+00,seconds
31,us-central1-c,market-1,default,memory,F449-33EC-A5EF,2021-11-19 18:29:53+00:00,2021-11-19 19:09:00+00:00,0.002446,16792662016,sporadic-customer,9.641927e+10,byte-seconds
32,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:30:00+00:00,2021-11-19 19:08:56+00:00,0.000761,4,sporadic-customer,7.110000e+00,seconds
33,us-central1-c,market-1,default,memory,F449-33EC-A5EF,2021-11-19 18:30:00+00:00,2021-11-19 19:08:56+00:00,0.002496,16792662016,sporadic-customer,9.791767e+10,byte-seconds
34,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:29:53+00:00,2021-11-19 19:08:56+00:00,0.000675,4,sporadic-customer,6.330000e+00,seconds
35,us-central1-c,market-1,default,memory,F449-33EC-A5EF,2021-11-19 18:29:53+00:00,2021-11-19 19:08:56+00:00,0.002465,16792662016,sporadic-customer,9.698329e+10,byte-seconds
36,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:29:57+00:00,2021-11-19 19:08:52+00:00,0.000578,4,sporadic-customer,5.400000e+00,seconds
37,us-central1-c,market-1,default,memory,F449-33EC-A5EF,2021-11-19 18:29:57+00:00,2021-11-19 19:08:52+00:00,0.002457,16792662016,sporadic-customer,9.634062e+10,byte-seconds


# Resources

In [79]:
def load_influx(num_customers='10', num_suppliers='10', supplier_behavior='lazy'): 
    # load data
    file_path = os.path.join(os.getcwd(), "demo_results", "raw_influx", f"{num_customers}-{num_suppliers}-{supplier_behavior}.csv")
    df = pd.read_csv(file_path)
    df['_time'] = pd.to_datetime(df['_time'])
    
    # filter data
    state = df[df['_field']=='state']
    state = state[state['entity'].isin(['customer', 'supplier'])]
    completed_uuids = state[state['_value']=='11']['uuid'].unique().tolist()
    print(f"Number of completed uuids: {len(completed_uuids)}")
    state = state[state['uuid'].isin(completed_uuids)]
    
    allocation = df[df['_field']=='allocation']
    return df, state, allocation

def time_sync(allocation):
    start_time = allocation['_time'].min()
    end_time = allocation['_time'].max()
    time_diffs = []
    for alloc in allocation['_value'].unique():
        temp = allocation[allocation['_value']==alloc]
        time_diff = (temp['_time'].max() - temp['_time'].min()).total_seconds()
        time_diffs.append(time_diff)
    return start_time, end_time, time_diffs

num_customers='10'
num_suppliers='10'
supplier_behavior='lazy'

df, state, allocation = load_influx(num_customers=num_customers, num_suppliers=num_suppliers, supplier_behavior=supplier_behavior)
start_time, end_time, time_diff = time_sync(allocation)
df.head(2)

Number of completed uuids: 20


,time_ms,_measurement,_field,_value,uuid,_time,entity
0,0.0,Events,sender,customer_2f38edec422a4381b3c2b7af8f43d444,customer_2f38edec422a4381b3c2b7af8f43d444_offer_0,2021-11-19 18:30:31.085070+00:00,customer
1,0.0,Events,state,1,customer_2f38edec422a4381b3c2b7af8f43d444_offer_0,2021-11-19 18:30:31.085070+00:00,customer


In [80]:
print(start_time, end_time)
time_diff

2021-11-19 18:30:41.625745+00:00 2021-11-19 18:37:53.433708+00:00


[431.29919,
 232.250495,
 428.093533,
 430.927579,
 426.241943,
 413.519224,
 406.375871,
 431.274305,
 431.493613,
 413.610801]

In [56]:
df['_field'].unique()

array(['sender', 'state', 'allocation'], dtype=object)

In [64]:
sender = df[df['_field']=='sender']
print(len(sender))
sender.head(2)

296


,time_ms,_measurement,_field,_value,uuid,_time,entity
0,0.000,Events,sender,customer_2f38edec422a4381b3c2b7af8f43d444,customer_2f38edec422a4381b3c2b7af8f43d444_offer_0,2021-11-19 18:30:31.085070+00:00,customer
2,57.493,Events,sender,customer_2018422e3635406fb7a2682844c2facf,customer_2018422e3635406fb7a2682844c2facf_offer_0,2021-11-19 18:30:31.142563+00:00,customer


In [60]:
allocation = df[df['_field']=='allocation']
print(len(allocation))
allocation.head(1)

179


,time_ms,_measurement,_field,_value,uuid,_time,entity
194,10540.675,Events,allocation,allocation_0,customer_f798855e4f05413099b350e48b42a229_offer_0,2021-11-19 18:30:41.625745+00:00,customer


In [73]:
allocation['_'].unique()

array(['customer', 'supplier'], dtype=object)

In [72]:
allocation[allocation['_value']=='allocation_1']

,time_ms,_measurement,_field,_value,uuid,_time,entity
200,10583.395,Events,allocation,allocation_1,supplier_97a7047cbfd244a8a23c56857b7961e9_offer_2,2021-11-19 18:30:41.668465+00:00,supplier
203,10583.441,Events,allocation,allocation_1,customer_d34630e7c20d449f845935290bd1cacf_offer_0,2021-11-19 18:30:41.668511+00:00,customer
229,10732.276,Events,allocation,allocation_1,supplier_97a7047cbfd244a8a23c56857b7961e9_offer_2,2021-11-19 18:30:41.817346+00:00,supplier
232,10733.285,Events,allocation,allocation_1,customer_d34630e7c20d449f845935290bd1cacf_offer_0,2021-11-19 18:30:41.818355+00:00,customer
354,73737.246,Events,allocation,allocation_1,supplier_97a7047cbfd244a8a23c56857b7961e9_offer_2,2021-11-19 18:31:44.822316+00:00,supplier
383,89109.401,Events,allocation,allocation_1,customer_d34630e7c20d449f845935290bd1cacf_offer_0,2021-11-19 18:32:00.194471+00:00,customer
412,159847.573,Events,allocation,allocation_1,customer_d34630e7c20d449f845935290bd1cacf_offer_0,2021-11-19 18:33:10.932643+00:00,customer
416,165552.394,Events,allocation,allocation_1,supplier_97a7047cbfd244a8a23c56857b7961e9_offer_2,2021-11-19 18:33:16.637464+00:00,supplier
417,165862.771,Events,allocation,allocation_1,customer_d34630e7c20d449f845935290bd1cacf_offer_0,2021-11-19 18:33:16.947841+00:00,customer
420,165863.176,Events,allocation,allocation_1,supplier_97a7047cbfd244a8a23c56857b7961e9_offer_2,2021-11-19 18:33:16.948246+00:00,supplier


In [54]:
state.head(2)

,time_ms,_measurement,_field,_value,uuid,_time,entity
1,0.000,Events,state,1,customer_2f38edec422a4381b3c2b7af8f43d444_offer_0,2021-11-19 18:30:31.085070+00:00,customer
3,57.493,Events,state,1,customer_2018422e3635406fb7a2682844c2facf_offer_0,2021-11-19 18:30:31.142563+00:00,customer


# Resources

In [48]:
start_time = df.groupby(['uuid']).min()['start_timestamp'].max()
end_time = df.groupby(['uuid']).max()['end_timestamp'].min()
run_time = (end_time - start_time).total_seconds()
print(f"start_time: {start_time}, end_time: {end_time}, run_time: {run_time}")

start_time: 2021-11-19 20:19:44.875112+00:00, end_time: 2021-11-19 20:22:32.152535+00:00, run_time: 167.277423


In [50]:
start_time = df.groupby(['uuid']).min()['start_timestamp'].max()
end_time = df.groupby(['uuid']).max()['end_timestamp'].min()
run_time = (end_time - start_time).total_seconds()
print(f"start_time: {start_time}, end_time: {end_time}, run_time: {run_time}")

start_time: 2021-11-19 20:19:44.875112+00:00, end_time: 2021-11-19 20:22:32.152535+00:00, run_time: 167.277423


In [ ]:
def time_sync(num_customers='10', num_suppliers='10', supplier_behavior='lazy'):
    df = load_influx(num_customers=num_customers, num_suppliers=num_suppliers, supplier_behavior=supplier_behavior)
    start_time = df.groupby(['uuid']).min()['start_timestamp'].max()
    end_time = df.groupby(['uuid']).max()['end_timestamp'].min()
    run_time = (end_time - start_time).total_seconds()
    print(f"start_time: {start_time}, end_time: {end_time}, run_time: {run_time}")
    return start_time, end_time, run_time

In [25]:
num_customers='10'
num_suppliers='10'
supplier_behavior='lazy'

file_path = os.path.join(os.getcwd(), "demo_results", "raw_consumption", f"{num_customers}-{num_suppliers}-{supplier_behavior}-consumption.csv")
usage = pd.read_csv(file_path)
usage.head(1)

,cluster_location,cluster_name,namespace,resource_name,sku_id,start_time,end_time,fraction,cloud_resource_size,labels,project,usage
0,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19T18:29:57Z,2021-11-19T19:08:54Z,0.00053,4,"{\n ""labels"": [{\n ""key"": ""io.kompose.serv...","{\n ""project"": {\n ""id"": ""ascendant-choir-...","{\n ""usage"": {\n ""amount"": ""4.949999999999..."


In [ ]:
def load_consumption(num_customers='10', num_suppliers='10', supplier_behavior='lazy'):
    file_path = os.path.join(os.getcwd(), "demo_results", "raw_consumption", f"{num_customers}-{num_suppliers}-{supplier_behavior}-consumption.csv")
    usage = pd.read_csv(file_path)
    
    usage['start_time'] = pd.to_datetime(usage['start_time'])
    usage['end_time'] = pd.to_datetime(usage['end_time'])

    service = []
    usage_val = []
    usage_unit = []
    for k, v in usage.iterrows():
        temp = json.loads(v['labels'])['labels']
        temp2 = {x['key']: x['value'] for x in temp}
        if 'io.kompose.service' in temp2.keys():
            service.append(temp2['io.kompose.service'])
        else:
            service.append('none')
        temp3 = json.loads(v['usage'])['usage']
        usage_val.append(float(temp3['amount']))
        usage_unit.append(temp3['unit'])

    usage['service'] = service
    usage['usage_val'] = usage_val
    usage['usage_unit'] = usage_unit
    
    

In [26]:
usage['start_time'] = pd.to_datetime(usage['start_time'])
usage['end_time'] = pd.to_datetime(usage['end_time'])

service = []
usage_val = []
usage_unit = []
for k, v in usage.iterrows():
    temp = json.loads(v['labels'])['labels']
    temp2 = {x['key']: x['value'] for x in temp}
    if 'io.kompose.service' in temp2.keys():
        service.append(temp2['io.kompose.service'])
    else:
        service.append('none')
    temp3 = json.loads(v['usage'])['usage']
    usage_val.append(float(temp3['amount']))
    usage_unit.append(temp3['unit'])

usage['service'] = service
usage['usage_val'] = usage_val
usage['usage_unit'] = usage_unit
usage.head(1)

,cluster_location,cluster_name,namespace,resource_name,sku_id,start_time,end_time,fraction,cloud_resource_size,labels,project,usage,service,usage_val,usage_unit
0,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:29:57+00:00,2021-11-19 19:08:54+00:00,0.00053,4,"{\n ""labels"": [{\n ""key"": ""io.kompose.serv...","{\n ""project"": {\n ""id"": ""ascendant-choir-...","{\n ""usage"": {\n ""amount"": ""4.949999999999...",allocator-service,4.95,seconds


In [27]:
usage['service'].unique()

array(['allocator-service', 'lazy-supplier', 'mediator-service',
       'sporadic-customer', 'verifier-service', 'none'], dtype=object)

In [66]:
temp = usage[(usage['resource_name']=='cpu') & (usage['service']=='sporadic-customer')]
temp

,cluster_location,cluster_name,namespace,resource_name,sku_id,start_time,end_time,fraction,cloud_resource_size,labels,project,usage,service,usage_val,usage_unit
28,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:29:52+00:00,2021-11-19 19:08:48+00:00,0.000388,4,"{\n ""labels"": [{\n ""key"": ""io.kompose.netw...","{\n ""project"": {\n ""id"": ""ascendant-choir-...","{\n ""usage"": {\n ""amount"": ""3.629999999999...",sporadic-customer,3.63,seconds
30,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:29:53+00:00,2021-11-19 19:09:00+00:00,0.000604,4,"{\n ""labels"": [{\n ""key"": ""io.kompose.netw...","{\n ""project"": {\n ""id"": ""ascendant-choir-...","{\n ""usage"": {\n ""amount"": ""5.669999999999...",sporadic-customer,5.67,seconds
32,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:30:00+00:00,2021-11-19 19:08:56+00:00,0.000761,4,"{\n ""labels"": [{\n ""key"": ""io.kompose.netw...","{\n ""project"": {\n ""id"": ""ascendant-choir-...","{\n ""usage"": {\n ""amount"": ""7.109999999999...",sporadic-customer,7.11,seconds
34,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:29:53+00:00,2021-11-19 19:08:56+00:00,0.000675,4,"{\n ""labels"": [{\n ""key"": ""io.kompose.netw...","{\n ""project"": {\n ""id"": ""ascendant-choir-...","{\n ""usage"": {\n ""amount"": ""6.329999999999...",sporadic-customer,6.33,seconds
36,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:29:57+00:00,2021-11-19 19:08:52+00:00,0.000578,4,"{\n ""labels"": [{\n ""key"": ""io.kompose.serv...","{\n ""project"": {\n ""id"": ""ascendant-choir-...","{\n ""usage"": {\n ""amount"": ""5.399999999999...",sporadic-customer,5.40,seconds
38,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:30:03+00:00,2021-11-19 19:08:49+00:00,0.000709,4,"{\n ""labels"": [{\n ""key"": ""io.kompose.netw...","{\n ""project"": {\n ""id"": ""ascendant-choir-...","{\n ""usage"": {\n ""amount"": ""6.599999999999...",sporadic-customer,6.60,seconds
40,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:29:53+00:00,2021-11-19 19:08:55+00:00,0.000810,4,"{\n ""labels"": [{\n ""key"": ""io.kompose.serv...","{\n ""project"": {\n ""id"": ""ascendant-choir-...","{\n ""usage"": {\n ""amount"": ""7.589999999999...",sporadic-customer,7.59,seconds
42,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:29:57+00:00,2021-11-19 19:08:51+00:00,0.000578,4,"{\n ""labels"": [{\n ""key"": ""io.kompose.netw...","{\n ""project"": {\n ""id"": ""ascendant-choir-...","{\n ""usage"": {\n ""amount"": ""5.399999999999...",sporadic-customer,5.40,seconds
44,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:29:52+00:00,2021-11-19 19:08:54+00:00,0.000371,4,"{\n ""labels"": [{\n ""key"": ""io.kompose.serv...","{\n ""project"": {\n ""id"": ""ascendant-choir-...","{\n ""usage"": {\n ""amount"": ""3.479999999999...",sporadic-customer,3.48,seconds
46,us-central1-c,market-1,default,cpu,CF4E-A0C7-E3BF,2021-11-19 18:29:59+00:00,2021-11-19 19:09:01+00:00,0.000500,4,"{\n ""labels"": [{\n ""key"": ""io.kompose.netw...","{\n ""project"": {\n ""id"": ""ascendant-choir-...","{\n ""usage"": {\n ""amount"": ""4.679999999999...",sporadic-customer,4.68,seconds


In [68]:
temp.iloc[0]['labels']

'{\n  "labels": [{\n    "key": "io.kompose.network/mv2-default",\n    "value": "true"\n  }, {\n    "key": "io.kompose.service",\n    "value": "sporadic-customer"\n  }, {\n    "key": "pod-template-hash",\n    "value": "b7466b74f"\n  }]\n}'

In [70]:
state[state['entity']=='customer']['uuid'].unique()

array(['customer_2f38edec422a4381b3c2b7af8f43d444_offer_0',
       'customer_2018422e3635406fb7a2682844c2facf_offer_0',
       'customer_bf1d8fdb610a4cd89dc8634dd319609e_offer_0',
       'customer_eac80ebaeeb445de95b054d7ea0fc502_offer_0',
       'customer_02654232d9014521baa848ee6bd95f62_offer_0',
       'customer_d34630e7c20d449f845935290bd1cacf_offer_0',
       'customer_7fb6f3e8d2fc4ac999cd55d4b7ec80d7_offer_0',
       'customer_f798855e4f05413099b350e48b42a229_offer_0',
       'customer_c1e4c42706fc437580b5b4b3b6858fe6_offer_0',
       'customer_18e24148e4334c1da241b9a868028f20_offer_0'], dtype=object)